In [158]:
import numpy as np
import scipy as sp
import pandas as pd

In [215]:
def read_file_data(filename: str, min_num_of_measurements=10) -> pd.DataFrame:
    file = open(filename, mode='r', encoding='utf-8-sig')
    lines = file.readlines()
    file.close()
    dict_of_results = {}
    my_list = []
    i = 0
    dict_blocks = {}

    for line in lines:
        if not line in ['\r\n', '\n', 'hello dwm1000!\n', 'init pass!\n', 'ERROR\n', 'AIT-BU01-DB V100 T2020-5-17\n', 'device:TAG ID:0\n']:

            if 'OK' in line:
                if dict_blocks:
                    name = list(dict_blocks.keys())[0]
                    if len(dict_blocks[name]) >= min_num_of_measurements:
                        dict_of_results[i] = dict_blocks.copy()
                        i += 1
                    dict_blocks.clear()

            else:
                line = line.split(':')
                line = [i.strip().replace('m', '') for i in line]
                if not line[0] in dict_blocks and len(line)>1:
                    dict_blocks[line[0]] = [float(line[1])]
                else:
                    dict_blocks[line[0]].append(float(line[1]))
    df_or_results = pd.DataFrame.from_dict(dict_of_results, orient="index")
    return df_or_results


def get_array_from_df(dictionary, n_values_to_drop=5):
    np_array = dictionary.to_numpy()
    index = range(n_values_to_drop - 1)
    return [[np.delete(np.array(i), index), np.delete(np.array(j), index)] for i, j in np_array]

In [216]:

def calc_precision(x: np.array):
    return np.std(x)


def calc_dist_to_tags(dist: float, tr_angle: float, dist_btw_tags: float, in_degrees=True) -> [float, float]:
    """
                ^
               /|\
      tr_side1/ | \tr_side2
             /  |d \
            /___|)__\
       l_tag  a ^    r_tag
                | mid_point_btw_tags
    :param dist: d
    :param tr_angle:
    :param dist_btw_tags:
    :param in_degrees: boolean, as default - True
    :return: tr_sides
    """

    if in_degrees:
        tr_angle = tr_angle * np.pi / 180
    # midpoint btw tags
    a = dist_btw_tags / 2
    tr_side2 = np.sqrt(a ** 2 + dist ** 2 - 2 * a * dist * np.cos(tr_angle))
    tr_side1 = np.sqrt(a ** 2 + dist ** 2 - 2 * a * dist * np.cos(np.pi - tr_angle))

    return tr_side1, tr_side2

In [223]:
def calc_results(filename: str, array_of_results: np.ndarray):
    filename = filename.split('/')[1]
    exp_info = filename.split("_")[:-2]
    angl_in_positive = 1
    if exp_info[0] == "min":
        angl_in_positive = -1
        exp_info.pop(0)
    angle, angle_units, dist = [f"{i} " for i in exp_info]
    print(f"Exp info:\n\tAndle\tDistance btw tags (cm)\n\t{angle}{angle_units}\t{dist}")

    angle = float(angle) * np.pi / 180 * angl_in_positive
    angle = np.pi / 2 - float(angle) * np.pi / 180
    dist = float(dist) / 100
    block_size = 0.6

    for n_block in range(len(results)):
        real_dist_to_anchor = block_size * (n_block + 2) * np.sin(angle)
        truth_dist_to_tags = calc_dist_to_tags(dist=real_dist_to_anchor,
                                               tr_angle=angle, dist_btw_tags=dist, in_degrees=False)

        print(f"\tResults for {real_dist_to_anchor} m:")
        for n_tag in range(len(array_of_results[n_block])):
            estimated_val = np.mean(array_of_results[n_block][n_tag])
            print(f"\tTag {n_tag + 1}")
            print(f"\t\tTrue dist:\t\t\t{truth_dist_to_tags[n_tag]} m")
            print(f"\t\tEstimated dist:\t{estimated_val} m ")
            print(f"\t\tStandard deviation:\t{calc_precision(array_of_results[n_block][n_tag]) * 100} cm")
            print(
                f"\t\tDifference between real data and measurement:\t{np.abs(truth_dist_to_tags[n_tag] - estimated_val) * 100} cm")
    calc_precision(array_of_results[0][1])

In [224]:
import os

# Get the list of all files and directories
path = "2_tags_1_anchor_measurements"
dir_list = os.listdir(path)

for file_name in dir_list:
    file_name = path + '/' + file_name
    df_of_results = read_file_data(file_name, min_num_of_measurements=20)
    results = get_array_from_df(df_of_results)
    calc_results(file_name, results)

2_tags_1_anchor_measurements/min_45_deg_60_5_blocks.txt
Exp info:
	Andle	Distance btw tags (cm)
	45 deg 	60 
	Results for 1.1998872597618606 m:
	Tag 1
		True dist:			1.2328264549585604 m
		Estimated dist:	0.9014545454545453 m 
		Standard deviation:	0.6719897677300488 cm
		Difference between real data and measurement:	33.137190950401504 cm
	Tag 2
		True dist:			1.2408053047243 m
		Estimated dist:	1.2646296296296295 m 
		Standard deviation:	1.7817278646901862 cm
		Difference between real data and measurement:	2.3824324905329597 cm
	Results for 1.7998308896427908 m:
	Tag 1
		True dist:			1.8206011861944549 m
		Estimated dist:	1.8841304347826084 m 
		Standard deviation:	11.64619005989311 cm
		Difference between real data and measurement:	6.352924858815356 cm
	Tag 2
		True dist:			1.828713696413975 m
		Estimated dist:	2.106521739130435 m 
		Standard deviation:	3.2115104891512014 cm
		Difference between real data and measurement:	27.780804271645998 cm
	Results for 2.3997745195237212 m:
	Tag 